## GiLOT

In [1]:
model_name = 'llama2-7b-chat'
beams = 200
max_new_tokens = 10

In [2]:
from utils import load_model, get_template
model, tokenizer, block_name, embedding_name, embed_token_name, _, _ = load_model(model_name)

/opt/conda/envs/llama_factory/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.17s/it]


{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 1, 'model.layers.4': 1, 'model.layers.5': 1, 'model.layers.6': 1, 'model.layers.7': 1, 'model.layers.8': 2, 'model.layers.9': 2, 'model.layers.10': 2, 'model.layers.11': 2, 'model.layers.12': 2, 'model.layers.13': 3, 'model.layers.14': 3, 'model.layers.15': 3, 'model.layers.16': 3, 'model.layers.17': 3, 'model.layers.18': 4, 'model.layers.19': 4, 'model.layers.20': 4, 'model.layers.21': 4, 'model.layers.22': 4, 'model.layers.23': 5, 'model.layers.24': 5, 'model.layers.25': 5, 'model.layers.26': 5, 'model.layers.27': 5, 'model.layers.28': 6, 'model.layers.29': 6, 'model.layers.30': 6, 'model.layers.31': 6, 'model.norm': 6, 'lm_head': 6}


In [3]:
from interpreter import Interpreter
interpreter = Interpreter(model, block_name, embed_token_name, embed_token_name)

In [4]:
from utils import load_model, get_template
template = get_template(model_name)

beams = 20
max_new_tokens = 20

query = "Give three tips for staying healthy."
input_text = f"{template['prefix']}{query.strip()}{template['postfix']}"
inputs = tokenizer(input_text, return_tensors="pt")
inputs.to(0)
attributions, probs_sequences = interpreter.interpret_ours(inputs.input_ids, beams, max_new_tokens, "optimal_transport")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


### Visualizations

In [6]:
from captum.attr import visualization

def show_heatmap_on_text(text, text_encoding, R_text):
    # CLS_idx = text_encoding.argmax(dim=-1)
    # print(R_text,  CLS_idx, type(R_text), type(CLS_idx))
    # R_text = R_text[CLS_idx, 1:CLS_idx]

    text_scores = R_text / R_text.sum()
    text_scores = text_scores.flatten()
    text_tokens = tokenizer.encode(text)
    text_tokens_decoded = [tokenizer.decode([a]) for a in text_tokens]
    vis_data_records = [visualization.VisualizationDataRecord(text_scores,0,0,0,0,0,text_tokens_decoded,1)]
    visualization.visualize_text(vis_data_records)

In [10]:
import torch
attribution_scores = torch.stack([d['optimal_transport'] for d in attributions])
show_heatmap_on_text(input_text, inputs.input_ids, attribution_scores)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,0.00,#s [ INST ] Give three tips for stay ing health y . [ / INST ]
